# Approach 2 - Fractal Hypothesis Knwoledge Graph and Fractal Embedding Space

In [2]:
from utils import feature_space_templatev1
from matscibert_inf.ner.NER_inference import NER_INF

/home/ppathak/miniconda3/envs/matscibert/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load models:

In [3]:
OBJ_ner_inf = NER_INF()
MODEL_ner = OBJ_ner_inf.initialize_infer()
PATH_model = "/home/ppathak/Hypothesis_Generation_Active_Learning/matscibert_inf/mlm_models/tr_150_tech/checkpoint-2940"
OBJ_2005_bert = feature_space_templatev1.FeatureSpaceSingleTemplate(OBJ_ner_inf)

OBJ_2005_bert.PATH_all_model = PATH_model
MODEL_2005_bert = OBJ_2005_bert.load_model()

/home/ppathak/miniconda3/envs/matscibert/lib/python3.9/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at m3rg-iitd/matscibert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ppathak/Hypothesis_Generation_Active_Learning/matscibert_inf/ner/NER_inference.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECU

## Steps of this approach:

1. Start with a sample Hypothesis
2. Understand the Main Keywords in play that makes up the important part of the hypothesis
3. Now form Sub-Hypothesis about these keywords.
4. Repeat the create a Fractal Hypothesis State.
5. Convert each hypothesis to embedding.
6. Experiment with above.

Starting with the following Hypothesis:

`An element [MASK] is an electrical conductor`

In [4]:
PRINCIPLE_HYPOTHESIS = "An element [MASK] is an electrical conductor"
hypothesis_fractal = {PRINCIPLE_HYPOTHESIS : {}}

## Running the Principle Hypothesis through NER to get word candidates to create Sub-Hypothesis'

In [5]:
ner_results = OBJ_ner_inf.infer_caption(PRINCIPLE_HYPOTHESIS, MODEL_ner)
ner_results

/home/ppathak/miniconda3/envs/matscibert/lib/python3.9/site-packages/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:530.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


{'an': 'O',
 'element': 'O',
 '[MASK]': 'O',
 'is': 'O',
 'electrical': 'B-PRO',
 'conductor': 'I-PRO'}

## Now here, we combine the B- and I- to take the full `action-word` that is impacting our hypothesis. `electrical conductor` in this case.

I used Chat-GPT to make further "Prompts" that are factually correct about the "action-word". I gave it the following promt:

>Here I will give a simple statement. And in that statement. I will highlight a keyword. And then I will give a list of simple questions to answer about that keyword in context to the sentence. You just have to answer those questions. The questions will always be about >Material Science Domain. While answering the questions Ensure the following:
>1. The answers need to be as short of as possible.
>2. Answers has to be direct.
>3. The answers has to clearly "mention" any property/material/phenomenon that needs to be included without providing and extra explanation about said terms.
>4. The answer should never talk about applications or about any other info/fact or place where the subject word was applied unless asked.

>This is your First Statement: "An element [MASK] is an electrical conductor"
>and your key word is "electrical conductor"

>For this, answer the following in context to the statement:

>What is the subject, define it?

>How is the subject related to in context of the statement, how does it impact it, use other scientific term to define the importance?

>Why is it important in context to the statement?

>How is it important in context to the statement?

Now that I have the answers of the above, I repeat the ner results for the same:

In [6]:
def remove_o_tag(result_dict: dict, sub_hypothesis_terms: dict):
    action_words = {}
    grouped_term = ""
    current_tag = None

    for word, tag in result_dict.items():
        if tag == 'O':
            # If we're in a group, finalize it and reset
            if grouped_term and current_tag:
                action_words[grouped_term] = current_tag
                if grouped_term + '_' + current_tag not in sub_hypothesis_terms:
                    sub_hypothesis_terms[grouped_term + '_' + current_tag] = {
                        grouped_term: current_tag
                    }
                grouped_term = ""
                current_tag = None
            continue

        prefix, label = tag.split('-') if '-' in tag else (None, tag)

        if prefix == 'B':  # Start of a new group
            # If there's an existing group, finalize it
            if grouped_term and current_tag:
                action_words[grouped_term] = current_tag
                if grouped_term + '_' + current_tag not in sub_hypothesis_terms:
                    sub_hypothesis_terms[grouped_term + '_' + current_tag] = {
                        grouped_term: current_tag
                    }
            # Start new group
            grouped_term = word
            current_tag = label
        elif prefix == 'I' and current_tag == label:  # Continuation of the current group
            grouped_term += f" {word}"
        else:
            # If `I-` is out of sync with the current tag, treat it as a new group
            if grouped_term and current_tag:
                action_words[grouped_term] = current_tag
                if grouped_term + '_' + current_tag not in sub_hypothesis_terms:
                    sub_hypothesis_terms[grouped_term + '_' + current_tag] = {
                        grouped_term: current_tag
                    }
            grouped_term = word
            current_tag = label

    # Finalize the last group
    if grouped_term and current_tag:
        action_words[grouped_term] = current_tag
        if grouped_term + '_' + current_tag not in sub_hypothesis_terms:
            sub_hypothesis_terms[grouped_term + '_' + current_tag] = {
                grouped_term: current_tag
            }

    return action_words


In [7]:
sub_hypothesis_terms = {}
WHAT_ner_result = "A material that permits the flow of electric charge, characterized by low electrical resistivity."
WHY_ner_result = "It determines the material's functionality as a pathway for electrical current."
HOW_ner_result = "The material's ability to conduct electricity depends on properties like free electron density, low band gap, and high conductivity."

In [8]:
WHAT_results = OBJ_ner_inf.infer_caption(WHAT_ner_result, MODEL_ner)
WHAT_results

{'a': 'O',
 'material': 'O',
 'that': 'O',
 'permits': 'O',
 'the': 'O',
 'flow': 'O',
 'of': 'O',
 'electric': 'O',
 'charge': 'O',
 ',': 'O',
 'characterized': 'O',
 'by': 'O',
 'low': 'O',
 'electrical': 'B-PRO',
 'resistivity': 'I-PRO',
 '.': 'O'}

In [9]:
# Clearning it a bit
WHAT_action_words = remove_o_tag(WHAT_results, sub_hypothesis_terms)
WHAT_action_words

{'electrical resistivity': 'PRO'}

In [10]:
WHY_results = OBJ_ner_inf.infer_caption(WHY_ner_result, MODEL_ner)
WHY_results

{'it': 'O',
 'determines': 'O',
 'the': 'O',
 'material': 'O',
 "'": 'O',
 's': 'O',
 'functionality': 'O',
 'as': 'O',
 'a': 'O',
 'pathway': 'O',
 'for': 'O',
 'electrical': 'O',
 'current': 'O',
 '.': 'O'}

In [11]:
# Clearning it a bit
WHY_action_words = remove_o_tag(WHY_results, sub_hypothesis_terms)
WHY_action_words

{}

In [12]:
HOW_results = OBJ_ner_inf.infer_caption(HOW_ner_result, MODEL_ner)
HOW_results

{'the': 'O',
 'material': 'O',
 "'": 'O',
 's': 'O',
 'ability': 'O',
 'to': 'O',
 'conduct': 'B-PRO',
 'electricity': 'I-PRO',
 'depends': 'O',
 'on': 'O',
 'properties': 'O',
 'like': 'O',
 'free': 'B-PRO',
 'electron': 'I-PRO',
 'density': 'I-PRO',
 ',': 'O',
 'low': 'O',
 'band': 'B-PRO',
 'gap': 'I-PRO',
 'and': 'O',
 'high': 'O',
 'conductivity': 'B-PRO',
 '.': 'O'}

In [13]:
# Clearning it a bit
HOW_action_words = remove_o_tag(HOW_results, sub_hypothesis_terms)
HOW_action_words

{'conduct electricity': 'PRO',
 'free electron density': 'PRO',
 'band gap': 'PRO',
 'conductivity': 'PRO'}

In [14]:
# Final terms for Principle_hypothesis:
sub_hypothesis_terms

{'electrical resistivity_PRO': {'electrical resistivity': 'PRO'},
 'conduct electricity_PRO': {'conduct electricity': 'PRO'},
 'free electron density_PRO': {'free electron density': 'PRO'},
 'band gap_PRO': {'band gap': 'PRO'},
 'conductivity_PRO': {'conductivity': 'PRO'}}

# - Starting with a new Hypothesis:

`An alloy composed of Element X and Element Y makes a good heat sink.`

Let's try the above approach with this hypothesis.

For each NER result we ask the following prompt with Chat-GPT:

>For the sentence "A `< NER-TERM >` composed of Element [MASK] and Element [MASK] makes a good heat sink". what does `< NER-TERM >` mean? Give short and direct answer.

In [15]:
PRINCIPLE_HYPOTHESIS = "An alloy composed of Element [MASK] and Element [MASK] makes a good heat sink"
hypothesis_fractal = {PRINCIPLE_HYPOTHESIS : {}}
ner_results = OBJ_ner_inf.infer_caption(PRINCIPLE_HYPOTHESIS, MODEL_ner)
new_sub_hypothesis_terms = {}
ner_results

{'an': 'O',
 'alloy': 'B-DSC',
 'composed': 'O',
 'of': 'O',
 'element': 'O',
 '[MASK]': 'B-MAT',
 'and': 'O',
 'makes': 'O',
 'a': 'O',
 'good': 'O',
 'heat': 'B-APL',
 'sink': 'I-APL'}

In [16]:
h1_action_words = remove_o_tag(ner_results, new_sub_hypothesis_terms)
h1_action_words
hypothesis_fractal[PRINCIPLE_HYPOTHESIS] = h1_action_words

In [18]:
alloy_prompt_output = 'In Material Science, an "alloy" is a mixture of two or more elements, at least one of which is a metal, designed to achieve specific properties like improved strength, corrosion resistance, or thermal conductivity.'
alloy_ner_results = OBJ_ner_inf.infer_caption(alloy_prompt_output, MODEL_ner)
h2_alloy_action_words = remove_o_tag(alloy_ner_results, new_sub_hypothesis_terms)
hypothesis_fractal[PRINCIPLE_HYPOTHESIS] = h2_alloy_action_words
h2_alloy_action_words

{'alloy': 'DSC',
 'strength': 'PRO',
 'corrosion resistance': 'PRO',
 'thermal conductivity': 'PRO'}

In [19]:
heat_sink_prompt_output = 'In Material Science, a "heat sink" is a material or device designed to absorb and dissipate heat efficiently, often used to manage thermal energy in electronic or mechanical systems.'
heat_sink_ner_results = OBJ_ner_inf.infer_caption(heat_sink_prompt_output, MODEL_ner)
h2_hs_action_words = remove_o_tag(heat_sink_ner_results, new_sub_hypothesis_terms)
hypothesis_fractal[PRINCIPLE_HYPOTHESIS] = h2_hs_action_words
h2_hs_action_words

{'sink': 'APL',
 'thermal energy': 'PRO',
 'electronic': 'APL',
 'mechanical systems': 'APL'}

## For H3 Hypothesis we merge the Principle Hypothesis and H2 together. This pattern will continue for downstream hypothesis as well.

The prompt now becomes:

`For the sentences "An alloy composed of Element [MASK] and Element [MASK] makes a good heat sink", "In Material Science, an "alloy" is a mixture of two or more elements, at least one of which is a metal, designed to achieve specific properties like improved strength, corrosion resistance, or thermal conductivity. what does "thermal conductivity" mean? Give short and direct answer.`

In [20]:
h3_thermal = 'In Material Science, "thermal conductivity" is the ability of a material to transfer heat through its structure.'
thermal_conductivitiy_ner_results = OBJ_ner_inf.infer_caption(h3_thermal, MODEL_ner)
h3_tc_action_words = remove_o_tag(thermal_conductivitiy_ner_results, new_sub_hypothesis_terms)
hypothesis_fractal[PRINCIPLE_HYPOTHESIS] = h3_tc_action_words
h3_tc_action_words

{'thermal conductivity': 'PRO', 'structure': 'PRO'}

## Doing the Above but automating it as an evolving dictionary with many steps:

> We will also ensure that this dictionary is ArangoDB Compatible to later visualize the `Fractal Hypothesis Knowledge Graph`.

In [21]:
from string import Template

In [22]:
PRINCIPLE_HYPOTHESIS = "An alloy composed of Element [MASK] and Element [MASK] makes a good heat sink"
PROMPT_TEMPALTE = Template('For the sentence "$Hypothesis". what does "$keyword" mean? Give short and direct answer.')

fractal_hypothesis_graph = {0 : {"H" : PRINCIPLE_HYPOTHESIS, "_keywords" : {}}}

In [ ]:
def update_graph(keyword:str, hypothesis:str):
    pass